In [2]:
import furniture_bench

import gym
import cv2
import torch
import numpy as np

#from arp.aloha.lerobot.common.utils.nn import SinusoidalPositionEmbedding2d

import sys
from arp.furni.dataset import FurnitureOfflineDataset
from tqdm import tqdm

import arp.aloha.arp as arp

In [3]:
import yaml

with open("arp/furni/configs/test0.yaml", "r") as f:
    full_config = yaml.load(f, Loader=yaml.FullLoader)

config = full_config["policy"]
arp_cfg = full_config["policy"]["arp_cfg"]

Instantiate arp policy. 

See https://github.com/mlzxy/arp/issues/9 for the meaning of tokentypes and how to do compute_loss

In [4]:
policy = arp.AutoRegressivePolicy(arp.ModelConfig(
    n_embd=arp_cfg["n_embd"],
    embd_pdrop=arp_cfg["embd_pdrop"],
    max_seq_len=config["chunk_size"],
    max_chunk_size=config["chunk_size"],
    layers=[
        arp.LayerType.make(**arp_cfg["layer_cfg"], condition_on='visual-tokens')
    ] * arp_cfg["num_layers"],
    tokens=[
        arp.TokenType.make(
            name='state_ee_position', is_continuous=True, dim=3, embedding='linear', predictor='gmm', 
            predictor_kwargs={'num_latents': config["num_latents"]}
        ),
        arp.TokenType.make(
            name='state_ee_quaternion', is_continuous=True, dim=4, embedding='linear', predictor='gmm', 
            predictor_kwargs={'num_latents': config["num_latents"]}
        ),
        arp.TokenType.make(
            name='state_ee_velocity', is_continuous=True, dim=3, embedding='linear', predictor='gmm', 
            predictor_kwargs={'num_latents': config["num_latents"]}
        ),
        arp.TokenType.make(
            name='state_ee_angular_velocity', is_continuous=True, dim=3, embedding='linear', predictor='gmm', 
            predictor_kwargs={'num_latents': config["num_latents"]}
        ),
        arp.TokenType.make(
            name='state_joint_positions', is_continuous=True, dim=7, embedding='linear', predictor='gmm', 
            predictor_kwargs={'num_latents': config["num_latents"]}
        ),
        arp.TokenType.make(
            name='state_joint_velocities', is_continuous=True, dim=7, embedding='linear', predictor='gmm', 
            predictor_kwargs={'num_latents': config["num_latents"]}
        ),
        arp.TokenType.make(
            name='state_joint_torques', is_continuous=True, dim=7, embedding='linear', predictor='gmm', 
            predictor_kwargs={'num_latents': config["num_latents"]}
        ),
        arp.TokenType.make(
            name='action_ee_delta_position', is_continuous=True, dim=3, embedding='linear', predictor='gmm', 
            predictor_kwargs={'num_latents': config["num_latents"]}
        ),
        arp.TokenType.make(
            name='action_ee_delta_quaternion', is_continuous=True, dim=4, embedding='linear', predictor='gmm', 
            predictor_kwargs={'num_latents': config["num_latents"]}
        ),
        arp.TokenType.make(
            name='action_gripper_range', is_continuous=True, dim=1, embedding='linear', predictor='gmm', 
            predictor_kwargs={'num_latents': config["num_latents"]}
        )
    ]
))

The following is a map from each token type's name to its id. This map is automatically generated for you once `policy` is instantiated.

In [5]:
policy.token_name_2_ids

{'state_ee_position': 0,
 'state_ee_quaternion': 1,
 'state_ee_velocity': 2,
 'state_ee_angular_velocity': 3,
 'state_joint_positions': 4,
 'state_joint_velocities': 5,
 'state_joint_torques': 6,
 'action_ee_delta_position': 7,
 'action_ee_delta_quaternion': 8,
 'action_gripper_range': 9}

In [6]:
device = "cuda:0"

Training loop pseudocode

In [ ]:
data_dir = "./furniture-bench/data/low_compressed/lamp"
dataset = FurnitureOfflineDataset(
    data_dir=data_dir,
    batch_size=16,
    seq_len=3
)

dataloader, sampler = dataset.dataloader()

for batch in tqdm(dataloader):
    batch_tuple = tuple(v for k, v in batch.items())
    tk_vals = arp.cat_uneven_blc_tensors(**batch_tuple)
    tk_ids = torch.as_tensor(tk_ids).to(device)[None, :, None].repeat(batch_size, 1, 1)
    tks = torch.cat([tk_vals, tk_ids], dim=-1) 
    chk_ids = torch.as_tensor(chk_ids, device=device)[None, :]

    loss_dict = self.policy.compute_loss(tks, chk_ids, contexts={ 'visual-tokens': encoder_out, 
            'visual-featmap': visual_featmap, 
            'smooth-heatmap-right': heatmap_right.flatten(0, 1), 
            'smooth-heatmap-left': heatmap_left.flatten(0, 1) }, valid_tk_mask=~tk_is_pad_mask)
    
    loss = sum(v for k, v in loss_dict.items())
    loss.backward()


# for batch in tqdm(dataloader):
#     if i == 0:
#         print(batch)
    #batch = {k:v.to(device) for k,v in batch.items()}